In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

sample = pd.read_csv('/content/drive/MyDrive/tmap/data/TMAP_data_sample_20220825.csv')
cal = pd.read_csv('/content/drive/MyDrive/tmap/data/calendar_20220825.csv')
submit = pd.read_csv('/content/drive/MyDrive/tmap/data/report_format.csv')

In [ ]:
# 회귀 결측치 대체 함수
def regression_nan(exdata):
    exdata = exdata.interpolate(method='linear', limit_direction = 'both', axis = 0)
    return exdata

In [ ]:
# 시간데이터 datetime 형식 변경해주기
sample['periodtime'] = pd.to_datetime(sample['periodtime'], format='%Y%m%d%H%M')

# 바꾼 데이터로 날짜와 시간 분리하기
## 날짜
sample['date'] = sample['periodtime'].dt.strftime('%Y%m%d')
## 시간
sample['hour'] = sample['periodtime'].dt.strftime('%H')

# 시간데이터 datetime 형식 변경해주고, format
cal['perioddate'] = pd.to_datetime(cal['perioddate'], format='%Y%m%d')
cal['date'] = cal['perioddate'].dt.strftime('%Y%m%d')

data = pd.merge(sample, cal, on='date')

data_1018052 = data[data['tsdlinkid']==1018052]
data_2594592 = data[data['tsdlinkid']==2594592]
# weekday 변수 추가 (금요일=4)
data_1018052['weekday'] = data_1018052['periodtime'].dt.weekday
data_2594592['weekday'] = data_2594592['periodtime'].dt.weekday

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# 1018052

In [ ]:
# 기본 데이터 전처리 함수
def data_processing(exdata):
    # 금요일인 날짜만 추출
    exdata = exdata[exdata['weekday']==4]
    # 6시 데이터만 추출
    exdata = exdata[exdata['hour']=='18']
    # 공휴일 데이터 제외
    exdata = exdata[exdata['description'].isnull()]
    # 시간데이터 datetime 형식 변경해주기
    exdata['periodtime'] = pd.to_datetime(exdata['periodtime'])
    # index를 periodtime으로 바꿔주기
    exdata = exdata.set_index('periodtime')
    # 평균 계산이 필요 없는 데이터 제거해주기
    del exdata['tsdlinkid']
    del exdata['length']
    del exdata['offerspeed']
    del exdata['offertraveltime']
    del exdata['hour']
    del exdata['perioddate']
    del exdata['holiday']
    del exdata['dayofweek']
    del exdata['description']
    del exdata['weekday']

    return exdata

In [ ]:
df_1018052 = data_processing(data_1018052)

In [ ]:
df_1018052 = regression_nan(df_1018052)

## pivot

In [ ]:
def pivot(exdata):
  exdata = exdata.copy()
  exdata = exdata.reset_index()
  exdata['periodtime'] = pd.to_datetime(exdata['periodtime'])
  exdata['time'] = exdata['periodtime'].dt.strftime('%H%M')
  exdata = exdata.pivot('time','date','truetraveltime')

  return exdata

In [ ]:
pivot_1018052 = pivot(df_1018052)

In [ ]:
# 6/3일
pivot_0603 = pivot_1018052.iloc[:,:14]
y_pred = pivot_1018052.mean(axis = 'columns')
y_true = pivot_1018052.iloc[:,14]

import numpy as np

def average(y_true,y_pred):
  return np.median(np.abs(y_true - y_pred)/y_true)

average(y_true,y_pred)

0.4698245247859883

In [ ]:
# 6/10일
pivot_0603 = pivot_1018052.iloc[:,:15]
y_pred = pivot_1018052.mean(axis = 'columns')
y_true = pivot_1018052.iloc[:,15]

import numpy as np

def average(y_true,y_pred):
  return np.median(np.abs(y_true - y_pred)/y_true)

average(y_true,y_pred)

In [ ]:
#pivot_1018052.to_csv('pivot_1018052.csv')

# 2594592

In [ ]:
df_2594592 = data_processing(data_2594592)

In [ ]:
df_2594592 = regression_nan(df_2594592)

In [ ]:
df_2594592

,truespeed,truetraveltime,date
periodtime,,,
2022-02-25 18:00:00,19.239155,449.08417,20220225
2022-02-25 18:01:00,19.264994,448.48184,20220225
2022-02-25 18:02:00,19.131395,451.61368,20220225
2022-02-25 18:03:00,18.990576,454.96250,20220225
2022-02-25 18:04:00,19.102911,452.28708,20220225
...,...,...,...
2022-06-10 18:55:00,13.102275,659.42740,20220610
2022-06-10 18:56:00,14.325012,603.14080,20220610
2022-06-10 18:57:00,15.228549,567.35540,20220610


## pivot

In [ ]:
pivot_2594592 = pivot(df_2594592)

In [ ]:
# 6/3일
pivot_0603 = pivot_2594592.iloc[:,:14]
y_pred = pivot_2594592.mean(axis = 'columns')
y_true = pivot_2594592.iloc[:,14]

import numpy as np

def average(y_true,y_pred):
  return np.median(np.abs(y_true - y_pred)/y_true)

average(y_true,y_pred)

0.14338739608926954

In [ ]:
# 6/10일
pivot_0603 = pivot_2594592.iloc[:,:15]
y_pred = pivot_2594592.mean(axis = 'columns')
y_true = pivot_2594592.iloc[:,15]

import numpy as np

def average(y_true,y_pred):
  return np.median(np.abs(y_true - y_pred)/y_true)

average(y_true,y_pred)

0.2721935970877669

In [ ]:
pivot_2594592

date,20220225,20220304,20220311,20220318,20220325,20220401,20220408,20220415,20220422,20220429,20220506,20220513,20220520,20220527,20220603,20220610
time,,,,,,,,,,,,,,,,
1800,449.08417,739.29736,534.018300,91.809875,575.24570,577.56030,819.809750,1204.346300,993.13000,643.00930,1029.93930,652.70530,559.999450,804.977000,478.28100,594.33276
1801,448.48184,739.29736,552.031700,90.476105,520.01370,590.84210,838.168760,1241.911900,978.84080,645.00006,1023.06586,664.98770,575.846740,794.870700,552.96950,625.65360
1802,451.61368,700.35156,532.706850,91.107864,469.22192,612.29480,871.365500,1244.055700,967.63000,645.97210,1027.54440,693.58765,631.589540,786.230350,562.07120,655.56630
1803,454.96250,624.15314,518.187440,91.460640,457.51334,622.84686,911.851560,1261.533200,963.16766,645.20776,1032.91980,701.46730,660.884340,780.905460,571.68120,716.56866
1804,452.28708,627.18380,500.927280,90.512180,430.13248,644.89667,938.262600,1274.410600,899.87415,639.11000,1032.04520,703.95410,692.240500,763.707760,605.46860,779.70807
1805,449.72780,626.59600,451.573500,91.626816,404.07593,663.24210,948.547800,1293.172700,899.87415,624.10706,1054.62870,705.29320,728.582760,761.836700,630.48505,804.73640
1806,442.57420,643.46783,410.016720,91.974170,371.29816,665.80524,939.202450,1295.003700,848.94720,600.14730,999.29224,706.98360,752.304930,732.102900,675.15630,822.96100
1807,421.90344,681.86865,388.623700,92.804140,343.91428,663.47340,911.242070,1286.721200,861.46893,551.45984,970.98160,685.91290,776.220200,690.990000,703.29170,827.08490
1808,394.39170,731.11850,372.976200,91.911705,342.02142,668.75640,902.129640,1263.889600,863.45170,522.59520,945.93190,656.36676,793.659550,654.539900,721.53796,870.29890


In [ ]:
#pivot_2594592.to_csv('pivot_2594592.csv')

# 모델링

## submit

### index별 평균

In [ ]:
submit_1018052 = submit[submit['tsdlinkid']==1018052]
submit_2594592 = submit[submit['tsdlinkid']==2594592]

In [ ]:
mean_1018052 = pivot_1018052.mean(axis = 'columns')
mean_2594592 = pivot_2594592.mean(axis = 'columns')

In [ ]:
mean_1018052 = mean_1018052.reset_index()
del mean_1018052['time']

In [ ]:
mean_2594592 = mean_2594592.reset_index()
del mean_2594592['time']

In [ ]:
submit_1018052['truetraveltime_predict'] = mean_1018052
submit_1018052.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,periodtime,tsdlinkid,truetraveltime_predict
0,202206171800,1018052,863.374388
1,202206171801,1018052,831.812889
2,202206171802,1018052,824.861200
3,202206171803,1018052,840.260610
4,202206171804,1018052,849.286086


In [ ]:
submit_2594592.to_csv('submit_2594592.csv')

In [ ]:
submit_2594592 = pd.read_csv('submit_2594592.csv')

In [ ]:
submit_2594592['truetraveltime_predict'] = mean_2594592
submit_2594592.head()

,Unnamed: 0,periodtime,tsdlinkid,truetraveltime_predict
0,60,202206171800,2594592,671.721617
1,61,202206171801,2594592,680.153652
2,62,202206171802,2594592,683.931838
3,63,202206171803,2594592,688.456929
4,64,202206171804,2594592,692.170067


In [ ]:
submit_2594592 = submit_2594592.iloc[:,1:4]

In [ ]:
submit_2594592.to_csv('submit_2594592.csv')

In [ ]:
submit_final = pd.concat([submit_1018052, submit_2594592], axis = 0)

In [ ]:
submit_final = submit_final.reset_index()

In [ ]:
submit_final.set_index('periodtime')

,index,tsdlinkid,truetraveltime_predict
periodtime,,,
202206171800,0,1018052,863.374388
202206171801,1,1018052,831.812889
202206171802,2,1018052,824.861200
202206171803,3,1018052,840.260610
202206171804,4,1018052,849.286086
...,...,...,...
202206171855,55,2594592,521.085900
202206171856,56,2594592,508.804411
202206171857,57,2594592,494.753373


In [ ]:
submit_v2 = submit_final.iloc[:,1:4]

In [ ]:
submit_v2

,periodtime,tsdlinkid,truetraveltime_predict
0,202206171800,1018052,863.374388
1,202206171801,1018052,831.812889
2,202206171802,1018052,824.861200
3,202206171803,1018052,840.260610
4,202206171804,1018052,849.286086
...,...,...,...
115,202206171855,2594592,521.085900
116,202206171856,2594592,508.804411
117,202206171857,2594592,494.753373
118,202206171858,2594592,475.858334


In [ ]:
submit_v2.to_csv('submit_v2.csv')